In [ ]:
from io import StringIO 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

In [ ]:
input_file = "data.csv"
dataset = pd.read_csv(input_file, header = 0)
d=dataset
input_file = "closed.dat"
b = pd.read_csv(StringIO(''.join(l.replace('\t',' ') for l in open(input_file))), header = None, sep=' ')
for name in ['4pir', '6be1', '6his','2bg9','3jad','6ubs','6huk','7koo']:
    c=b[b.eq(name).any(axis=1)].iloc[:,1:4]
    d=pd.concat([d,c])
d.iloc[:40,11:14]=d.iloc[-40:,11:14]
input_file = "open.dat"
b = pd.read_csv(StringIO(''.join(l.replace('\t',' ') for l in open(input_file))), header = None, sep=' ')
for name in ['6dg8','6hin','6y5a','7kox','6ud3','3jae']:
    c=b[b.eq(name).any(axis=1)].iloc[:,1:4]
    d=pd.concat([d,c])
d.iloc[40:70,11:14]=d.iloc[110:141,11:14]
d=d.rename(columns={1:'M1/M2',2:'M1/M3',3:'M2/M3'})
e = d[['PDB','M1 radial tilt','M1 tangential tilt','M2 radial tilt','M2 tangential tilt','M3 radial tilt','M3 tangential tilt','ECD TMD twist','Subunit TMD interface contact area','M1/M3 M2 interface contact area','M1/M2','M1/M3','M2/M3','Class']]
dataset=e[:70]
dataset

In [ ]:
X = dataset.iloc[:,1:13].values
y = dataset.iloc[:,13].values
y = y.astype(int)
np.shape(X)

In [ ]:
prec_acc=[]
precsd_acc=[]
size_acc=[]
impr_acc=[]
imprsd_acc=[]
aucc_acc=[]
rfc_acc=[]
for perc in [0.1, 0.3, 0.5]: 
    precision=[]
    importance=[]
    auc=[]
    for _ in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=perc, random_state=None)
        rfc = RandomForestClassifier(n_estimators=100, random_state=None)
        rfc.fit(X_train, y_train)
        preds=rfc.predict_proba(X_test)
        a=y_test
        a=np.where((a==0)|(a==1), a^1, a)
        precision.append((np.sum(preds[:,1]*y_test)+np.sum(preds[:,0]*a))/np.size(y_test))
        importance.append(rfc.feature_importances_)
    impr_acc.append(np.mean(np.stack(importance), axis=0))
    imprsd_acc.append(np.std(np.stack(importance), axis=0))
    size_acc.append(np.size(y_test))
    prec_acc.append(np.mean(precision))
    precsd_acc.append(np.std(precision))
    rfc_acc.append(rfc)

In [ ]:
plt.rcParams.update({'font.size': 24})
fig, ax = plt.subplots()
patch = ax.bar(np.shape(X)[0]-np.array(size_acc),prec_acc, 10, yerr=precsd_acc, capsize=10)
patch[0].set_facecolor('#94B49F')
patch[1].set_facecolor('#DF7861')
patch[2].set_facecolor('#3A5BA0')
plt.xticks(np.shape(X)[0]-np.array(size_acc))
plt.yticks(np.arange(0,1.1,.2))
plt.show()

In [ ]:
plt.rcParams.update({'font.size': 16})
labels=['Size 1', 'Size 2', 'Size 3']
x=np.arange(1,13)
fig, ax = plt.subplots()
ax.bar(x+.2,impr_acc[0], .2, yerr=imprsd_acc[2], capsize=2, facecolor='#94B49F')
ax.bar(x,impr_acc[1], .2, yerr=imprsd_acc[1], capsize=2, facecolor='#DF7861')
ax.bar(x-.2,impr_acc[2], .2, yerr=imprsd_acc[0], capsize=2, facecolor='#3A5BA0')
plt.xticks(x)
plt.show()

In [ ]:
prec_acc=[]
size_acc=[]
impr_acc=[]
aucc_acc=[]
rfc_acc=[]
labels=['Test 1', 'Test 2', 'Test 3', 'Test 4', 'Test 5']
for start in np.arange(0,70,5): 
    precision=[]
    importance=[]
    auc=[]
    for _ in range(1):
        X_train=np.delete(X,np.arange(start,start+5),0)
        y_train=np.delete(y,np.arange(start,start+5),0)
        X_test=X[start:start+5]
        y_test=y[start:start+5]
        rfc = RandomForestClassifier(n_estimators=100, random_state=0)
        rfc.fit(X_train, y_train)
        preds=rfc.predict_proba(X_test)
        a=y_test
        a=np.where((a==0)|(a==1), a^1, a)
        precision.append((np.sum(preds[:,1]*y_test)+np.sum(preds[:,0]*a))/np.size(y_test))
        importance.append(rfc.feature_importances_)
        print(preds)
        fig, ax = plt.subplots(figsize=(5,3))  
        ax.barh(labels,-preds[:,0],0.3)
        ax.barh(labels,preds[:,1],0.3)
        ax.barh(labels,preds[:,1]*y_test-preds[:,0]*a,0.4,facecolor='white',edgecolor='k',lw=3,alpha=.3)
        plt.show()
    impr_acc.append(np.mean(np.stack(importance), axis=0))
    size_acc.append(np.size(y_test))
    prec_acc.append(np.mean(precision))
    rfc_acc.append(rfc)
labels=['4PIR', '6BE1', '6HIS', '2BG9', '3JAD', '6UBS', '6HUK', '7KOO', '6DG8', '6HIN', '6Y5A', '7KOX', '6UD3', '3JAE']
fig, ax = plt.subplots()
patch = ax.bar(labels,prec_acc, .5)
plt.vlines(x=8.5,ymin=0,ymax=1, color='k', linestyle='--')
plt.xticks(rotation = 75)
plt.show()

In [ ]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
def stratified_kfold_score(clf,X,y,n_fold):
    strat_kfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=None)
    accuracy_list = []
    for train_index, test_index in strat_kfold.split(X, y):
        x_train_fold, x_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]
        clf.fit(x_train_fold, y_train_fold)
        preds = clf.predict_proba(x_test_fold)
        a=y_test_fold
        a=np.where((a==0)|(a==1), a^1, a)
        accuracy_list.append((np.sum(preds[:,1]*y_test_fold)+np.sum(preds[:,0]*a))/np.size(y_test_fold))
    return np.array(accuracy_list).mean()
prec_acc=[]
size_acc=[]
impr_acc=[]
aucc_acc=[]
rfc_acc=[]
for perc,nfold in zip([0.1, 0.3, 0.5],[10,3,2]): 
    precision=[]
    importance=[]
    auc=[]
    for _ in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=perc, random_state=None)
        def bo_params_rf(n_estimators):
            params = {
                'n_estimators':int(n_estimators)
            }
            clf = RandomForestClassifier(n_estimators=params['n_estimators'])
            score = stratified_kfold_score(clf,X,y,nfold)
            return score
        rf_bo = BayesianOptimization(bo_params_rf, {
                                              'n_estimators':(100,999)
                                             })
        rf_bo.maximize(n_iter=3, init_points=3)
        params = rf_bo.max['params']
        params['n_estimators']= int(params['n_estimators'])
        rfc = RandomForestClassifier(n_estimators=params['n_estimators'])
        rfc.fit(X_train, y_train)
        preds=rfc.predict_proba(X_test)
        a=y_test
        a=np.where((a==0)|(a==1), a^1, a)
        precision.append((np.sum(preds[:,1]*y_test)+np.sum(preds[:,0]*a))/np.size(y_test))
        importance.append(rfc.feature_importances_)
        auc.append(roc_auc_score(y_test,preds[:,1]))
    impr_acc.append(np.mean(np.stack(importance), axis=0))
    size_acc.append(np.size(y_test))
    prec_acc.append(np.mean(precision))
    aucc_acc.append(np.mean(auc))
    rfc_acc.append(rfc)